In [ ]:
import tensorflow as tf
import tflearn
import numpy as np
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from random import shuffle

In [ ]:
IMG_SIZE = 54 # 50 x 50 image, for example
TIME_LEN = 10 # 5 time points, for example
LR = 1e-3 # learning rate, 1e-3 for example
CONV_LAYERS = 13
POOL_LAYERS = 5
NO_CHANNELS = 1 # how many input streams i.e. a network with 3 channels could have RGB, depth, and body skeleton streams
VERSION = 'VGG-1' # to keep track of changes in a particular model
NO_SIGNS = 3 # number of signs to learn - i.e. the alphabet
N_EPOCH = 100 # number of epochs
MODEL_PATH = '3dconv-{}--{}--{}--{}--{}'.format(LR, CONV_LAYERS, POOL_LAYERS, NO_CHANNELS, VERSION)
MODEL_NAME = '3dconv-{}--{}--{}--{}--{}-.model'.format(LR, CONV_LAYERS, POOL_LAYERS, NO_CHANNELS, VERSION)

In [ ]:
conv_net = input_data(shape = [None, TIME_LEN, IMG_SIZE, IMG_SIZE, NO_CHANNELS], name = 'input')

conv_net = conv_3d(conv_net, 64, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 64, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = max_pool_3d(conv_net, 2, strides = 2)

conv_net = conv_3d(conv_net, 128, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 128, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = max_pool_3d(conv_net, 2, strides = 2)

conv_net = conv_3d(conv_net, 256, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 256, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 256, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = max_pool_3d(conv_net, 2, strides = 2)

conv_net = conv_3d(conv_net, 512, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 512, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 512, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = max_pool_3d(conv_net, 2, strides = 2)

conv_net = conv_3d(conv_net, 512, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 512, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = conv_3d(conv_net, 512, 3, strides = 1, padding = 'same', activation = 'relu')
conv_net = max_pool_3d(conv_net, 2, strides = 2)

conv_net = fully_connected(conv_net, 4096, activation='relu')
conv_net = dropout(conv_net, 0.5)
conv_net = fully_connected(conv_net, 4096, activation='relu')
conv_net = dropout(conv_net, 0.5)
conv_net = fully_connected(conv_net, 3, activation='softmax')

conv_net = regression(conv_net, optimizer = 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')


In [ ]:
train_data = np.load("trainingData.npy")
train_data_extra = np.load("trainingDataRotated.npy")
train_data_all = list(np.concatenate((train_data, train_data_extra), axis = 0))
shuffle(train_data_all)

X = np.array([i[0] for i in train_data_all]).reshape([-1, TIME_LEN, IMG_SIZE, IMG_SIZE, 1])
Y = [i[1] for i in train_data_all]

In [ ]:
model = tflearn.DNN(conv_net, tensorboard_dir = 'log', checkpoint_path = MODEL_PATH, max_checkpoints = 1, tensorboard_verbose = 1)
model.fit(X, Y, n_epoch = N_EPOCH, validation_set = 0.1, show_metric = True, run_id = MODEL_PATH, snapshot_step = 100, shuffle = True) 


In [ ]:
model.save(MODEL_NAME)